## Notes 

- rens van der schoot
- 


## Ideas

- symantic web technology, micro applications (combine the approaches)
- the role of authors, citations
- sensitivity 1 doesn't work. (multiple raters, different expertise)
- citation networks
- use of rules how to interpreted the text. 
- 






## Remarks

- Covidence and Rayyan (software)
- 



# Systematic review

Based on: https://github.com/keras-team/keras/blob/master/examples/pretrained_word_embeddings.py

> This script loads pre-trained word embeddings (GloVe embeddings)
into a frozen Keras Embedding layer, and uses it to
train a text classification model on the 20 Newsgroup dataset
(classification of newsgroup messages into 20 different categories).
GloVe embedding data can be found at:
http://nlp.stanford.edu/data/glove.6B.zip
(source page: http://nlp.stanford.edu/projects/glove/)
20 Newsgroup data can be found at:
http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.html


## Related links
https://machinelearningmastery.com/develop-bidirectional-lstm-sequence-classification-python-keras/
https://machinelearningmastery.com/best-practices-document-classification-deep-learning/
https://www.quora.com/What-deep-learning-method-to-use-to-classify-text-files

Import dependencies

In [1]:
%matplotlib inline

In [2]:
from __future__ import print_function

import os
import sys
import numpy as np
import pandas as pd
import sklearn
import keras_metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D, LSTM
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras import metrics
from keras.callbacks import TensorBoard

from nltk.corpus import stopwords
from prettytable import PrettyTable
from numpy import cumsum
from matplotlib import pyplot
from pandas import DataFrame
from datetime import datetime
from scipy import stats
from keras.layers import Bidirectional

Using TensorFlow backend.


## Project properties

Basic output properties related to the folder structure.

In [3]:
OUTPUT_DIR = os.path.join("..", "output")

LOG_DIR = os.path.join("logs")

## Data

Basic data related variables.

In [4]:
# the data folder
BASE_DIR = os.path.join("..", "data")

# the folder with the word2vec 
GLOVE_DIR = os.path.join(BASE_DIR)

# the papers about PTSD
TEXT_DATA_DIR = os.path.join(BASE_DIR, "example_dataset_1", "csv")

# the target variable
TARGET_VARIABLE = "included_final" # "included_ats" (after title screening)


Read data in memory.

In [5]:
# second, prepare text samples and their labels
print('Processing text dataset')

data_path = os.path.join(TEXT_DATA_DIR, "schoot-lgmm-ptsd-traindata.csv")
full_data = pd.read_csv(data_path)

texts = (full_data['title'].fillna('') + ' ' + full_data['abstract'].fillna(''))
labels = full_data[TARGET_VARIABLE]
print('Found %s texts.' % len(texts))

Processing text dataset
Found 5077 texts.


## Word tokenizing

Apply vectorization to texts.

In [6]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000

In [7]:
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# get the word index
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, 
                     maxlen=MAX_SEQUENCE_LENGTH, 
                     padding='post', truncating='post')
labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Found 26396 unique tokens.
Shape of data tensor: (5077, 1000)
Shape of label tensor: (5077, 2)


## Create training and validation set
Split the dataset into a training and a validation set. Use SciKit-learn for this. 
We decided to train our model on a small dataset (around 500), to make the environment more simulated to the active learning approach where we have small number of labeled data points. 

In [8]:
from sklearn.model_selection import train_test_split

def split_data(validation_split, added_positives):
    x_train, x_val, y_train, y_val = train_test_split(
        data,
        labels,
        test_size=validation_split,
        random_state=2018,
        stratify=labels
    )

    # add added_positives positive paper to training dataset
    if added_positives>0 :
        positive_indx = np.where(y_val[:,1]==1)[0]
        x_train = np.vstack((x_train, x_val[positive_indx[0:added_positives]]))
        y_train = np.vstack((y_train,y_val[positive_indx[0:added_positives]]))

        x_val = np.delete(x_val, positive_indx[0:added_positives],0)
        y_val = np.delete(y_val, positive_indx[0:added_positives],0)

    return (x_train, x_val, y_train, y_val)

x_train, x_val, y_train, y_val = split_data(0.2, 0)

print("x_train shape:", x_train.shape, ", x_val shape:", x_val.shape)
print("y_train shape:", y_train.shape, ", y_val shape:", y_val.shape)
print((y_train[:,1]==1).sum())
print((y_val[:,1]==1).sum())


x_train shape: (4061, 1000) , x_val shape: (1016, 1000)
y_train shape: (4061, 2) , y_val shape: (1016, 2)
32
8


We added 15 positive cases to training, it causes that model label all data as positive. Now I try with stratified samples.

## Prepare embedding layer


In [9]:
## set dimensions
EMBEDDING_DIM = 300

In [10]:
# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors.')

embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'wiki.en.vec'), encoding='utf8') as f:
    for line in f:
        
        values = line.split()
        split_on_i = len(values) - EMBEDDING_DIM
        word = ' '.join(values[0:split_on_i])
        coefs = np.asarray(values[split_on_i:], dtype='float32')
        embeddings_index[word] = coefs
print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 2518927 word vectors.


In [11]:
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
            continue
        
    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
                
print('Shape of embedding matrix: ', embedding_matrix.shape)

Shape of embedding matrix:  (20000, 300)


## Make an Keras embedding layer.

In [12]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
def build_embedding(weights):
    return Embedding(num_words,
                                EMBEDDING_DIM,
                                weights=[weights],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=False)

embedding_layer = build_embedding(embedding_matrix)

## Models

This section contains models to use later on. Focusing on Convolutional and LSTM models. 

### LSTM

In [13]:
def get_lstm_model(backwards, dropout):
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    x = LSTM(10,input_shape=(MAX_SEQUENCE_LENGTH,),  go_backwards=backwards, dropout=dropout)(embedded_sequences)
    x = Dense(128, activation='relu')(x)
    output = Dense(2, activation='softmax')(x)

    model_lstm = Model(inputs=sequence_input, outputs=output)

    model_lstm.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['acc',keras_metrics.recall()])

    model_lstm.summary()
    return model_lstm

def get_bi_lstm_model(mode, dropout):
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    x = Bidirectional(LSTM(10,input_shape=(MAX_SEQUENCE_LENGTH,), dropout=dropout),merge_mode= mode)(embedded_sequences)
    x = Dense(128, activation='relu')(x)
    output = Dense(2, activation='softmax')(x)

    model_lstm = Model(inputs=sequence_input, outputs=output)

    model_lstm.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['acc',keras_metrics.recall()])

    model_lstm.summary()
    return model_lstm


### Convnet

In [ ]:
# # train a 1D convnet with global maxpooling
# def build_convnet(embedded_sequences):
#     x = Conv1D(128, 5, activation='relu')(embedded_sequences)
#     x = MaxPooling1D(5)(x)
#     x = Conv1D(128, 5, activation='relu')(x)
#     x = MaxPooling1D(5)(x)
#     x = Conv1D(128, 5, activation='relu')(x)
#     x = GlobalMaxPooling1D()(x)
#     x = Dense(128, activation='relu')(x)
#     preds = Dense(2, activation='softmax')(x)

#     model_convnet = Model(inputs=sequence_input, outputs=preds)

#     model_convnet.compile(loss='binary_crossentropy',
#                   optimizer='rmsprop',
#                   metrics=['acc',keras_metrics.recall()])
#     return model_convnet
    
    
# sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

# ##build convnet model
# embedded_sequences = embedding_layer(sequence_input)
# model_convnet = build_convnet(embedded_sequences)

## Train the model

### calculating class weights

In [14]:
weights = [1/y_val[:, 0].mean(), 1/y_val[:, 1].mean()]
weights

[1.0079365041220509, 127.00000047311187]

### Visualize model scores with TensorBoard

In [15]:
def save_run(sub_dir=None):
    
    if sub_dir is not None:
        log_dir = os.path.join(LOG_DIR, sub_dir)
    else:
        log_dir = LOG_DIR
    
    return TensorBoard(log_dir=log_dir, histogram_freq=0,
                       write_graph=True, write_images=True)
    


In [18]:
def train_model(model, epoch_no):
    
    # fit model for one epoch on this sequence
    start=datetime.now()
    hist = model.fit(x_train, y_train,
            batch_size=BATCH_SIZE,
            epochs=epoch_no,
            validation_data=(x_val, y_val),
            shuffle=True,
            class_weight=weights,
            verbose = 0
        )  

    runtime=datetime.now() - start
    
    loss = hist.history['loss'][0]    
    return (model,loss, runtime)
 

def get_scores(model, threshhold):
    prediction = model.predict(x_val)
    y_classes = ( [0 if x < threshhold else 1 for x in prediction[:,1]])
    y_value = y_val.argmax(axis=1)
    tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_value,y_classes).ravel()
    return (tn, fp, fn, tp)


def get_pred(model):
    prediction = model.predict(x_val)
    return prediction


def get_scores_pred(pred, threshhold):
    y_classes = ( [0 if x < threshhold else 1 for x in pred[:,1]])
    y_value = y_val.argmax(axis=1)
    (tn, fp, fn, tp) = sklearn.metrics.confusion_matrix(y_value,y_classes).ravel()
    return (tn, fp, fn, tp)


### Compare Various LSTM models

In [19]:
epochs = 5
BATCH_SIZE = 128
dropout=0

In [20]:
# lstm forwards
print('lstm forwards')
model = get_lstm_model(False,dropout)
(trained_model,loss, runtime) = train_model(model, epochs)
forward_pred = get_pred(trained_model)
print('loss:' ,loss,'runtime:', runtime.total_seconds())

lstm forwards
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 300)         6000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 10)                12440     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1408      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 258       
Total params: 6,014,106
Trainable params: 14,106
Non-trainable params: 6,000,000
_________________________________________________________________
loss: 0.486743384675 runtime: 260.565955


In [21]:
print(stats.describe(forward_pred[:,1]))
threshhold=0.1
(tn, fp, fn, tp) = get_scores_pred(forward_pred, threshhold)
print(tn, fp, fn, tp)
forward_result = ('forward',threshhold,tn, fp, fn, tp, loss, runtime.total_seconds())

DescribeResult(nobs=1016, minmax=(0.0016443522, 0.0069454229), mean=0.0069402051, variance=2.765881e-08, skewness=-31.82767677307129, kurtosis=1011.0009158555156)
1008 0 8 0


In [22]:
# lstm backwards
print('lstm_back')
model = get_lstm_model(True,dropout)
(trained_model,loss, runtime) = train_model(model, epochs)
backward_pred = get_pred(trained_model)
print('loss:' ,loss,'runtime:', runtime.total_seconds())

lstm_back
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 300)         6000000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 10)                12440     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               1408      
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 258       
Total params: 6,014,106
Trainable params: 14,106
Non-trainable params: 6,000,000
_________________________________________________________________
loss: 0.209033353033 runtime: 359.102995


In [23]:
print(stats.describe(backward_pred[:,1]))
threshhold=0.009
(tn, fp, fn, tp) = get_scores_pred(backward_pred, threshhold)
print(tn, fp, fn, tp)
backward_result = ('backward',threshhold,tn, fp, fn, tp, loss, runtime.total_seconds())

DescribeResult(nobs=1016, minmax=(0.0005646699, 0.10106423), mean=0.0054643559, variance=5.9410402e-05, skewness=6.789302349090576, kurtosis=65.37682054588791)
872 136 3 5


In [24]:
# bidirectional concat
print('bidirectional concat')
model = get_bi_lstm_model('concat', 0)
(trained_model,loss, runtime) = train_model(model, epochs)
bilstm_pred = get_pred(trained_model)
print('loss:' ,loss,'runtime:', runtime.total_seconds())

bidirectional concat
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 300)         6000000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 20)                24880     
_________________________________________________________________
dense_5 (Dense)              (None, 128)               2688      
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 258       
Total params: 6,027,826
Trainable params: 27,826
Non-trainable params: 6,000,000
_________________________________________________________________
loss: 0.141877302439 runtime: 510.6483


In [27]:
print(stats.describe(bilstm_pred[:,1]))
threshhold=0.008
(tn, fp, fn, tp) = get_scores_pred(bilstm_pred, threshhold)
print(tn, fp, fn, tp)
bi_result = ('bidirectional',threshhold,tn, fp, fn, tp, loss, runtime.total_seconds())

# # line plot of results
# results.plot()
# pyplot.show()

t = PrettyTable(['Model','threshhold','TN', 'FP', 'FN','TP','Loss','RunTime (sec)'])

t.add_row(forward_result)
t.add_row(backward_result)
t.add_row(bi_result)

print(t)

DescribeResult(nobs=1016, minmax=(0.00039808519, 0.069011562), mean=0.0047325562, variance=2.0205429e-05, skewness=7.7373785972595215, kurtosis=85.67902693392917)
928 80 6 2
+---------------+------------+------+-----+----+----+----------------+---------------+
|     Model     | threshhold |  TN  |  FP | FN | TP |      Loss      | RunTime (sec) |
+---------------+------------+------+-----+----+----+----------------+---------------+
|    forward    |    0.1     | 1008 |  0  | 8  | 0  | 0.486743384675 |   260.565955  |
|    backward   |   0.009    | 872  | 136 | 3  | 5  | 0.209033353033 |   359.102995  |
| bidirectional |   0.008    | 928  |  80 | 6  | 2  | 0.141877302439 |    510.6483   |
+---------------+------------+------+-----+----+----+----------------+---------------+


### Setting probability threshhold 

- minimize number of papers
- max # of fn =1
- threshhold >0
x_train, x_val, y_train, y_val = split_data(0.8, 0)

print("x_train shape:", x_train.shape, ", x_val shape:", x_val.shape)
print("y_train shape:", y_train.shape, ", y_val shape:", y_val.shape)
print((y_train[:,1]==1).sum())
print((y_val[:,1]==1).sum())


### Comparing various LSTM models

Comparing LSTM forward, LSTM backward and Bi-directional LSTM

In [ ]:
results = DataFrame()
# sum merge
model = get_bi_lstm_model(embedded_sequences, 'sum')
results['bilstm_sum'] = train_model(model, 4)
# mul merge
model = get_bi_lstm_model(embedded_sequences, 'mul')
results['bilstm_mul'] = train_model(model, 4)
# avg merge
model = get_bi_lstm_model(embedded_sequences, 'ave')
results['bilstm_ave'] = train_model(model, 4)
# concat merge
model = get_bi_lstm_model(embedded_sequences, 'concat')
results['bilstm_con'] = train_model(model, 4)
# line plot of results
results.plot()
pyplot.show()

## Grid Search on hyperparameters

In [50]:
from datetime import datetime
from keras.layers import Bidirectional


def test_bi_lstm_dropout(epoch, dropout):
    
    model = get_bi_lstm_model('concat', dropout)
    (trained_model,loss, runtime) = train_model(model, epoch)
    bidirectional_pred = get_pred(trained_model)
    bidirectional_scores = (epoch, dropout,loss, runtime.total_seconds())
    return(bidirectional_pred,bidirectional_scores)
    
      
        
def run_test(epoch):        
    dropouts=[0.1,0.2,0.3,0.4,0.5,0.7]
     
    
    t = PrettyTable(['Epoch','Dropout','Loss','RunTime (sec)'])
    
    scores= list()
    preds = list()
    for d in dropouts:
        
        print('dropout:', d)
        (pred, score) = test_bi_lstm_dropout(epoch, d)
        scores.append(score)
        preds.append(pred)
        t.add_row(score)
    
    print(t) 
    return (preds,scores)  


In [25]:
 (preds,scores) = run_test(5)

dropout: 0.1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 300)         6000000   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 20)                24880     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               2688      
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 258       
Total params: 6,027,826
Trainable params: 27,826
Non-trainable params: 6,000,000
_________________________________________________________________
dropout: 0.2
_________________________________________________________________
Layer (type)                 Output

In [48]:
# stats.describe(preds[:,1][0])



print('dropout=0.1')
print(stats.describe(preds[0][:,1]))
threshhold=0.03
(tn, fp, fn, tp) = get_scores_pred(preds[0], threshhold)
print(tn, fp, fn, tp)


print('dropout=0.5')
print(stats.describe(preds[4][:,1]))
threshhold=0.03
(tn, fp, fn, tp) = get_scores_pred(preds[4], threshhold)
print(tn, fp, fn, tp)


print('dropout=0.7')
print(stats.describe(preds[5][:,1]))
threshhold=0.03
(tn, fp, fn, tp) = get_scores_pred(preds[5], threshhold)
print(tn, fp, fn, tp)


dropout=0.1
DescribeResult(nobs=4556, minmax=(0.016743055, 0.10690393), mean=0.036164369, variance=8.8121218e-05, skewness=0.8448181748390198, kurtosis=2.6091010298200237)
1185 3350 5 16
dropout=0.5
DescribeResult(nobs=4556, minmax=(0.015808305, 0.12366454), mean=0.041557342, variance=0.00016014803, skewness=0.9866886734962463, kurtosis=2.3124359111347177)
878 3657 3 18
dropout=0.7
DescribeResult(nobs=4556, minmax=(0.013476559, 0.079365678), mean=0.030714523, variance=7.7325727e-05, skewness=0.8100983500480652, kurtosis=1.0534353684582793)
2496 2039 9 12


In [51]:
(preds,scores) = run_test(10)

dropout: 0.1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 300)         6000000   
_________________________________________________________________
bidirectional_8 (Bidirection (None, 20)                24880     
_________________________________________________________________
dense_15 (Dense)             (None, 128)               2688      
_________________________________________________________________
dense_16 (Dense)             (None, 2)                 258       
Total params: 6,027,826
Trainable params: 27,826
Non-trainable params: 6,000,000
_________________________________________________________________
dropout: 0.2
_________________________________________________________________
Layer (type)                 Output

In [58]:
print('dropout=0.1')
print(stats.describe(preds[0][:,1]))
threshhold=0.07
(tn, fp, fn, tp) = get_scores_pred(preds[0], threshhold)
print(tn, fp, fn, tp)


dropout=0.1
DescribeResult(nobs=4556, minmax=(0.020900372, 0.17854363), mean=0.051863007, variance=0.00028814061, skewness=1.7441517114639282, kurtosis=6.015028404743033)
4002 533 12 9


In [68]:
print('dropout=0.3')
print(stats.describe(preds[2][:,1]))
threshhold=0.02
(tn, fp, fn, tp) = get_scores_pred(preds[2], threshhold)
print(tn, fp, fn, tp)


dropout=0.3
DescribeResult(nobs=4556, minmax=(0.0074478509, 0.096369863), mean=0.032401193, variance=6.4377266e-05, skewness=1.1691046953201294, kurtosis=4.129928662552521)
155 4380 0 21


In [64]:
print('dropout=0.7')
print(stats.describe(preds[5][:,1]))
threshhold=0.03
(tn, fp, fn, tp) = get_scores_pred(preds[5], threshhold)
print(tn, fp, fn, tp)


dropout=0.7
DescribeResult(nobs=4556, minmax=(0.007370112, 0.062455419), mean=0.029001787, variance=4.1923089e-05, skewness=0.47635743021965027, kurtosis=0.46874233525733633)
2670 1865 9 12
